# Analyse Gevorderd

1. Lees de wedstrijden 


In [ ]:
def lees_wedstrijden(bestandsnaam, delimiter):
    wedstrijden = []
    with open(bestandsnaam, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=delimiter)
        for row in reader:
            wedstrijden.append(row)
    return wedstrijden

2. Bereken de reguliere competitie punten

In [ ]:
def bereken_punten(wedstrijden, seizoen):
    punten = {}
    for wedstrijd in wedstrijden:
        if wedstrijd['Seizoen'] != seizoen:
            continue # Sla deze wedstrijd over als het niet overeenkomt met het opgegeven seizoen
        
        thuisploeg = wedstrijd['RoepnaamThuisploeg']
        uitploeg = wedstrijd['RoepnaamUitploeg']
        finale_stand_thuis = float(wedstrijd['FinaleStandThuisploeg'])
        finale_stand_uit = float(wedstrijd['FinaleStandUitploeg'])
        
        if thuisploeg not in punten:
            punten[thuisploeg] = 0
        if uitploeg not in punten:
            punten[uitploeg] = 0
        
        if finale_stand_thuis > finale_stand_uit:
            punten[thuisploeg] += 3
        elif finale_stand_thuis == finale_stand_uit:
            punten[thuisploeg] += 1
            punten[uitploeg] += 1
        else:
            punten[uitploeg] += 3
    
    return punten

3. Bereken play-offs 1 extra punten

In [ ]:
def bereken_playoff_punten(punten, playoffs, seizoen):
    """Berekent de extra punten voor elke ploeg op basis van de Play-off wedstrijden van het type 'Playoff_1'."""
    # Halveer de punten van reguliere competitie
    halve_punten = {ploeg: punten[ploeg] / 2 for ploeg in punten}
    
    # Bereken de extra punten voor elke ploeg op basis van de Play-off wedstrijden van het type 'Playoff_1'
    for playoff in playoffs:
        if playoff['seizoen_alternatief'] != seizoen or playoff['type'] != 'Playoff_1':
            continue # Sla deze wedstrijd over als het niet overeenkomt met het opgegeven seizoen of als het niet Play-off 1 is
        
        thuisploeg = playoff['roepnaam_thuis']
        uitploeg = playoff['roepnaam_uit']
        stand_thuis = int(playoff['stand_thuis'])
        stand_uit = int(playoff['stand_uit'])
        
        # Controleer of de ploegen al in de dictionary staan, zo niet, initialiseer met 0
        if thuisploeg not in halve_punten:
            halve_punten[thuisploeg] = 0
        if uitploeg not in halve_punten:
            halve_punten[uitploeg] = 0
        
        # Ken punten toe op basis van de uitslag
        if stand_thuis > stand_uit:
            halve_punten[thuisploeg] += 3
        elif stand_thuis == stand_uit:
            halve_punten[thuisploeg] += 1
            halve_punten[uitploeg] += 1
        else:
            halve_punten[uitploeg] += 3
    
    return halve_punten

4. Toont het resultaat en roept volgende functies op.

In [ ]:
def toon_resultaat(punten):
    """Toont de berekende punten voor elke ploeg."""
    # Sorteer op dalende orde
    sorted_punten = sorted(punten.items(), key=lambda x: x[1], reverse=True)
    
    print("STAND JUPILER PRO LEAGUE PLAYOFF I")
    print("#\tClub\t\tW\tG\tV\tDoelpunten\t+/-\tPnt.")
    for index, (ploeg, punten) in enumerate(sorted_punten, start=1):
        print(f"{index}\t{ploeg}\t\t\t\t\t\t{math.ceil(punten)}")

# Lees de reguliere competitie wedstrijden uit het CSV-bestand met puntkomma als scheidingsteken
wedstrijden = lees_wedstrijden('clean_matches.csv', ';')

# Bereken de punten voor elke ploeg voor het seizoen 21/22
punten = bereken_punten(wedstrijden, "21/22")

# Lees de playoff wedstrijden uit het CSV-bestand met puntkomma als scheidingsteken
playoffs = lees_wedstrijden('wedstrijd_playoffs.csv', ',')

# Bereken de extra punten voor elke ploeg voor de playoffs van het seizoen 21/22
extra_punten = bereken_playoff_punten(punten, playoffs, "21/22")

# Toon de resultaten
toon_resultaat(extra_punten)